In [17]:
import pandas as pd
import os

In [19]:
# Définir le chemin vers votre dossier
DATA_PATH = r"C:/Users/Etudiant/Documents/Cours M2 IDMC/Bussiness Intelligent BI/TPPOWERBIAFRIQUE"

In [21]:
# Chemin complet vers le fichier (adaptez si nécessaire)
file_path = os.path.join(DATA_PATH, 'Annex_FR_Tab1.xlsx')

In [23]:
# Vérifier si le fichier existe
if not os.path.exists(file_path):
    print(f"Erreur: Le fichier n'existe pas à {file_path}")
    print("Solution: Copiez 'Annex_FR_Tab1.xlsx' dans le dossier {DATA_PATH}")
    print("Ou téléchargez-le depuis l'attachement de la conversation et placez-le là.")
else:
    print(f"Fichier trouvé: {file_path}")

Fichier trouvé: C:/Users/Etudiant/Documents/Cours M2 IDMC/Bussiness Intelligent BI/TPPOWERBIAFRIQUE\Annex_FR_Tab1.xlsx


In [25]:
    # Charger le fichier Excel
    df = pd.read_excel(file_path, sheet_name='Tab1')

In [43]:
 # Nettoyer et renommer les colonnes
df = df.dropna(subset=[df.columns[1]])  # Supprimer lignes vides
df = df.reset_index(drop=True)
df.columns = ['Index', 'Code', 'Pays', 'Population', 'Superficie', 'Densite_Pop', 'PIB', 'PIB_Hab', 'Taux_Croissance']

In [47]:
 # Convertir colonnes numériques
numeric_cols = ['Population', 'Superficie', 'Densite_Pop', 'PIB', 'PIB_Hab', 'Taux_Croissance']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [49]:
 # 1. Informations générales (.info())
print("Informations générales sur le DataFrame:")
print(df.info())
    

Informations générales sur le DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            54 non-null     float64
 1   Code             54 non-null     object 
 2   Pays             54 non-null     object 
 3   Population       54 non-null     float64
 4   Superficie       54 non-null     float64
 5   Densite_Pop      54 non-null     float64
 6   PIB              53 non-null     float64
 7   PIB_Hab          53 non-null     float64
 8   Taux_Croissance  52 non-null     float64
dtypes: float64(7), object(2)
memory usage: 3.9+ KB
None


In [51]:
 # 2. Statistiques descriptives (.describe())
print("\nStatistiques descriptives:")
print(df[numeric_cols].describe())


Statistiques descriptives:
          Population   Superficie  Densite_Pop           PIB       PIB_Hab  \
count      54.000000    54.000000    54.000000  5.300000e+01     53.000000   
mean    21046.778278   556.785713    91.358608  1.024991e+05   5692.837299   
std     30343.715629   603.376750   121.042272  2.178346e+05   6829.575613   
min        93.306000     0.460000     2.848497  6.121399e+02    607.457921   
25%      2757.105250    60.527500    18.653502  1.018207e+04   1545.776604   
50%     11891.308000   332.230000    51.284001  2.710100e+04   2664.710007   
75%     23383.629500   898.900000    91.119590  7.195162e+04   6840.907755   
max    178516.904000  2381.740000   612.328922  1.057831e+06  32556.999754   

       Taux_Croissance  
count         52.00000  
mean           4.67125  
std            2.05069  
min           -1.30000  
25%            3.27500  
50%            4.40000  
75%            6.30000  
max           10.59000  


In [53]:
 # 3. Valeurs manquantes (.isnull().sum())
print("\nValeurs manquantes par colonne:")
print(df.isnull().sum())


Valeurs manquantes par colonne:
Index              0
Code               0
Pays               0
Population         0
Superficie         0
Densite_Pop        0
PIB                1
PIB_Hab            1
Taux_Croissance    2
dtype: int64


In [65]:
# 4. Détection des valeurs aberrantes (IQR)
print("\nDétection des valeurs aberrantes (méthode IQR):")
outliers_count = {}
outliers_data = {}
for col in numeric_cols:
    if df[col].dtype in ['float64', 'int64'] and df[col].notna().sum() > 0:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = ((df[col] < lower_bound) | (df[col] > upper_bound))
            count = outliers.sum()
            outliers_count[col] = count
            if count > 0:
                outliers_data[col] = df[outliers][['Pays', col]]
print(f"{col}: {count} valeur(s) aberrante(s)")


Détection des valeurs aberrantes (méthode IQR):
Taux_Croissance: 1 valeur(s) aberrante(s)


In [67]:
# Afficher lignes avec aberrantes
for col, data in outliers_data.items():
    print(f"\nLignes avec valeurs aberrantes dans '{col}':")
    print(data)
    


Lignes avec valeurs aberrantes dans 'Population':
                Pays  Population
12  Congo, Rép. dém.   69360.118
15           Égypte*   83386.739
18          Éthiopie   96506.031
37           Nigéria  178516.904

Lignes avec valeurs aberrantes dans 'Superficie':
                Pays  Superficie
0            Algérie     2381.74
12  Congo, Rép. dém.     2344.86

Lignes avec valeurs aberrantes dans 'Densite_Pop':
                    Pays  Densite_Pop
5                Burundi   376.670931
10               Comores   404.319183
32               Maurice   612.328922
38                Rwanda   459.379233
39  Sao Tomé-et-Principe   206.127083
41            Seychelles   202.839130

Lignes avec valeurs aberrantes dans 'PIB':
              Pays           PIB
0          Algérie  5.517202e+05
1           Angola  1.755401e+05
15         Égypte*  9.453878e+05
33           Maroc  2.543621e+05
37         Nigéria  1.057831e+06
44  Afrique du Sud  6.831468e+05

Lignes avec valeurs aberrantes dans 'PIB

In [69]:
# Aperçu final
print("\nAperçu du DataFrame nettoyé:")
print(df.head())
print(f"\nForme finale: {df.shape}")


Aperçu du DataFrame nettoyé:
   Index Code          Pays  Population  Superficie  Densite_Pop  \
0    1.0  DZA       Algérie   39928.947     2381.74    16.764612   
1    2.0  AGO        Angola   22137.261     1246.70    17.756686   
2    3.0  BEN         Bénin   10599.510      114.76    92.362409   
3    4.0  BWA      Botswana    2038.587      581.73     3.504353   
4    5.0  BFA  Burkina Faso   17419.615      274.22    63.524232   

             PIB       PIB_Hab  Taux_Croissance  
0  551720.200103  13817.549461              2.8  
1  175540.071705   7929.620187              7.0  
2   19846.755671   1872.421996              4.3  
3   33622.482561  16493.032949              4.8  
4   30080.577790   1726.822194              6.1  

Forme finale: (54, 9)
